# leetcode_wk4
146. LRU缓存机制
287. 寻找重复数
974. 和可被 K 整除的子数组
239. 滑动窗口最大值
230. 二叉搜索树中第K小的元素
329. 矩阵中的最长递增路径
147. 对链表进行插入排序
837. 新21点
344. 反转字符串
24. 两两交换链表中的节点
118. 杨辉三角
91. 解码方法
779. 第K个语法符号
39. 组合总和
52. N皇后 II
567. 字符串的排列
43. 字符串相乘
128. 最长连续序列
300. 最长上升子序列
516. 最长回文子序列
1466. 重新规划路线
126. 单词接龙 II
71. 简化路径
93. 复原IP地址

## 146. LRU缓存机制
运用你所掌握的数据结构，设计和实现一个  LRU (最近最少使用) 缓存机制。它应该支持以下操作： 获取数据 get 和 写入数据 put 。

获取数据 get(key) - 如果密钥 (key) 存在于缓存中，则获取密钥的值（总是正数），否则返回 -1。
写入数据 put(key, value) - 如果密钥已经存在，则变更其数据值；如果密钥不存在，则插入该组「密钥/数据值」。当缓存容量达到上限时，它应该在写入新数据之前删除最久未使用的数据值，从而为新的数据值留出空间。


进阶:

你是否可以在 O(1) 时间复杂度内完成这两种操作？

示例:

LRUCache cache = new LRUCache( 2 /* 缓存容量 */ );

cache.put(1, 1);
cache.put(2, 2);
cache.get(1);       // 返回  1
cache.put(3, 3);    // 该操作会使得密钥 2 作废
cache.get(2);       // 返回 -1 (未找到)
cache.put(4, 4);    // 该操作会使得密钥 1 作废
cache.get(1);       // 返回 -1 (未找到)
cache.get(3);       // 返回  3
cache.get(4);       // 返回  4


In [ ]:
#1.HashMap+双向链表
#260 ms	22.8 MB
class Node:
    def __init__(self,key,value):
        self.val = value
        self.key = key
        self.pre = None
        self.next = None
'''
hashmap存储key-value，key对应的value是链表中的node，用hashmap随机在0(1)内访问链表位置
'''
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.node_dict = {}
        self.head = None #头指针是LRU，即最久没操作过的
        self.tail = None #尾指针是最近一次操作放进来的

    def get(self, key: int) -> int:
        #有key的话把node挪到链表尾部
        if key in self.node_dict:
            cur = self.node_dict[key]
            if cur.next:#没在链表尾部，需要挪动
                if not cur.pre:#前边也没东西——在头上了
                    self.head = self.head.next#头指针从cur上挪开
                    self.head.pre = None #该节点左侧没东西了，清干净
                else:#节点在链表中间
                    cur.pre.next = cur.next#左侧的节点连到右侧的上——跳空
                    cur.next.pre = cur.pre#右侧的pre也连回来
                #这一套操作下cur已经被架空了,把它放到尾部8
                self.tail.next = cur#先放到当前tail的右侧
                cur.pre = self.tail
                cur.next = None
                self.tail = cur#然后更新tail
            return cur.val
        else:
            return -1

    def put(self, key: int, value: int) -> None:
        #在写入新数据之前删除最久未使用的数据值
        if key in self.node_dict:
            cur = self.node_dict[key]
            cur.val = value#更新节点的值，同时还得cur放到尾巴上去
            if cur.next:
                if not cur.pre:
                    self.head = self.head.next
                    self.head.pre = None
                else:
                    cur.pre.next = cur.next
                    cur.next.pre = cur.pre
                self.tail.next = cur
                cur.pre = self.tail
                cur.next = None
                self.tail = cur
            return
        if len(self.node_dict)==self.capacity:
            self.node_dict.pop(self.head.key)#头部的pop掉，LRU
            self.head = self.head.next
            if self.head:
                self.head.pre = None
            else:#缓存清空了
                self.tail = None
        self.node_dict[key]=Node(key,value)
        cur = self.node_dict[key]
        if self.tail:
            self.tail.next = cur
            cur.pre = self.tail
        else:#空缓存头一个，没有pre和next，不用连
            self.head = cur
        self.tail = cur
        
#2.纯队列
#4308 ms	22.1 MB
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.stack = []

    def get(self, key: int) -> int:
        for i in range(len(self.stack)):
            if key == self.stack[i][0]:
                tmp = self.stack[i]
                self.stack.pop(i)
                self.stack.append(tmp)
                #保持使用序
                return tmp[1]
        return -1

    def put(self, key: int, value: int) -> None:
        #在写入新数据之前删除最久未使用的数据值
        for i in range(len(self.stack)):
            if key==self.stack[i][0]:
                self.stack.pop(i)
                break

        if len(self.stack)==self.capacity:
            self.stack.pop(0)
        self.stack.append([key,value])

#3.Python自带字典有序
#280 ms	22.2 MB
class LRUCache:

    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}

    def get(self, key: int) -> int:
        if key in self.cache:
            self.cache[key]=self.cache.pop(key)#一次pop一次append,顺序更新了
            return self.cache[key]
        return -1

    def put(self, key: int, value: int) -> None:
        #在写入新数据之前删除最久未使用的数据值
        if key in self.cache:
            self.cache.pop(key)
        if len(self.cache)==self.capacity:
            self.cache.pop(list(self.cache)[0])
            #print(list(self.cache))
            #看样子默认是调用了dict的key，in也是这个道理
        self.cache[key]=value



## 287. 寻找重复数

In [3]:
#1.二分
#140 ms	16 MB
class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        #1.排序和swap被禁止，暴力不可，外数组字典被禁止
        #二分,这是因为限定1 to n
        l= 1
        r = len(nums)-1
        while l<r:
            mid = (r-l)//2+l
            cnt=0
            for i in range(len(nums)):
                if nums[i]<=mid:
                    cnt+=1
            if cnt<=mid:
                #重复值在右
                l = mid+1
            else:
                r = mid
        return r
#2.链表+快慢指针
#100 ms	16.1 MB
'''
快慢指针，一个时间复杂度为O(N)的算法。

其一，对于链表问题，使用快慢指针可以判断是否有环。

其二，本题可以使用数组配合下标，抽象成链表问题。但是难点是要定位环的入口位置。

举个例子：nums = [2,5, 9 ,6,9,3,8, 9 ,7,1]，构造成链表就是：2->[9]->1->5->3->6->8->7->[9]，也就是在[9]处循环。

其三，快慢指针问题，会在环内的[9]->1->5->3->6->8->7->[9]任何一个节点追上，不一定是在[9]处相碰，事实上会在7处碰上。

其四，必须另起一个for循环定位环入口位置[9]。这里需要数学证明。

对“其四”简单说明一下，既然快慢指针在环内的某处已经相碰了。那么，第二个for循环遍历时，res指针还是在不停的绕环走，但是必定和i指针在环入口处相碰。
'''
class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        #1.排序和swap被禁止，暴力不可，外数组字典被禁止
        #因为有重复数字的出现, 所以这个游走必然是成环的, 环的入口就是重复的元素
        fast = 0
        slow = 0
        while True:
            #不停跑下去直到撞上然后return
            fast = nums[nums[fast]]#快指针跑两次，跑到val为index的地方去
            slow = nums[slow]#慢指针跑一次
            if fast == slow:#初次相遇，因为有环所以会相遇
                fast = 0 #这块慢指针归0也一样
                while nums[slow]!=nums[fast]:
                    fast = nums[fast]
                    slow = nums[slow]
                return nums[slow]#找到了环的入口
'''
将 slow 放置起点后移动相遇的点就一定是答案了。假设环长为 L，从起点到环的入口的步数是 a，从环的入口继续走 b 步到达相遇位置，
从相遇位置继续走 c 步回到环的入口，则有 b+c=L、，其中 L、a、b、c 都是正整数。根据上述定义，慢指针走了 a+b 步，快指针走了 2(a+b) 步。
从另一个角度考虑，在相遇位置，快指针比慢指针多走了若干圈，因此快指针走的步数还可以表示成 a+b+kL，其中 k 表示快指针在环上走的圈数。
联立等式，可以得到2(a+b)=a+b+kL。解得 a=kL-b，整理可得
a=(k-1)L+(L-b)=(k-1)L+c
从上述等式可知，如果慢指针从起点出发，快指针从相遇位置出发，每次两个指针都移动一步，
则慢指针走了 a 步之后到达环的入口，快指针在环里走了 k−1 圈之后又走了 c 步，由于从相遇位置继续走 c 步即可回到环的入口，
因此快指针也到达环的入口。两个指针在环的入口相遇，相遇点就是答案。

'''

0


## 974. 和可被 K 整除的子数组
给定一个整数数组 A，返回其中元素之和可被 K 整除的（连续、非空）子数组的数目。

示例：

输入：A = [4,5,0,-2,-3,1], K = 5
输出：7
解释：
有 7 个子数组满足其元素之和可被 K = 5 整除：
[4, 5, 0, -2, -3, 1], [5], [5, 0], [5, 0, -2, -3], [0], [0, -2, -3], [-2, -3]
 

提示：

1 <= A.length <= 30000
-10000 <= A[i] <= 10000
2 <= K <= 10000

In [7]:
class Solution:
    def subarraysDivByK(self, A: List[int], K: int) -> int:
        '''
        通常，涉及连续子数组问题的时候，我们使用前缀和来解决。
        连续子数组的和就可以表示为前缀和的差
        两个数的差能被K整除，就说明这两个数 mod K得到的结果相同
        在保证余数相同的情况下，取出两个前缀和s可以得到一组答案,这两个数的差能被K整除。
        '''
        #前缀和之差
        s = [0]*(len(A)+1)#计算前缀和
        kcnt = [0]*K #modK余i
        for i in range(len(A)):
            #计算前缀和
            s[i+1]=s[i]+A[i]
        for summ in s:
            #modK余i
            kcnt[summ%K] +=1
        return sum(x*(x-1)//2 for x in kcnt) #这是在计算C_kcnt2

False


## 242. 有效的字母异位词
给定两个字符串 s 和 t ，编写一个函数来判断 t 是否是 s 的字母异位词。

示例 1:

输入: s = "anagram", t = "nagaram"
输出: true
示例 2:

输入: s = "rat", t = "car"
输出: false
说明:
你可以假设字符串只包含小写字母。

进阶:
如果输入字符串包含 unicode 字符怎么办？你能否调整你的解法来应对这种情况？

In [ ]:
class Solution:
    def isAnagram(self, s: str, t: str) -> bool:
        tmp = [0]*26
        if len(s)!=len(t):
            return False
        for i in range(len(s)):
            tmp[ord(s[i])-ord('a')]+=1
            tmp[ord(t[i])-ord('a')]-=1
        for i in range(len(tmp)):
            if tmp[i]!=0:
                return False
        return True
'''
Python里定义不定长或者空hashmap之后要调用的情况
1.dicts = collections.defaultdict(int)
  dicts[s[i]] = dicts[s[i]] + 1
2.d = {}
  d[c] = d.get(c, 0) + 1
'''

## 239. 滑动窗口最大值
给定一个数组 nums，有一个大小为 k 的滑动窗口从数组的最左侧移动到数组的最右侧。你只可以看到在滑动窗口内的 k 个数字。滑动窗口每次只向右移动一位。
返回滑动窗口中的最大值。
进阶：
你能在线性时间复杂度内解决此题吗？

示例:

输入: nums = [1,3,-1,-3,5,3,6,7], 和 k = 3
输出: [3,3,5,5,6,7] 
解释: 
```python
  滑动窗口的位置                最大值
---------------               -----
[1  3  -1] -3  5  3  6  7       3
 1 [3  -1  -3] 5  3  6  7       3
 1  3 [-1  -3  5] 3  6  7       5
 1  3  -1 [-3  5  3] 6  7       5
 1  3  -1  -3 [5  3  6] 7       6
 1  3  -1  -3  5 [3  6  7]      7
```
提示：

1 <= nums.length <= 10^5
-10^4 <= nums[i] <= 10^4
1 <= k <= nums.length

In [ ]:
#1.deque
#136 ms	22.7 MB
from collections import deque
class Solution:
    def maxSlidingWindow(self, nums: 'List[int]', k: 'int') -> 'List[int]':
        # base cases
        n = len(nums)
        if n * k == 0:
            return []
        if k == 1:
            return nums
        
        def clean_deque(i):
            #deque是用来记录下标的
            # remove indexes of elements not from sliding window，下标在滑窗外
            if deq and deq[0] == i - k:
                deq.popleft()
                
            # remove from deq indexes of all elements 
            # which are smaller than current element nums[i]
            while deq and nums[i] > nums[deq[-1]]:
                deq.pop()
        
        # init deque and output
        deq = deque()
        max_idx = 0
        for i in range(k):
            clean_deque(i)
            deq.append(i)
            # compute max in nums[:k]
            if nums[i] > nums[max_idx]:
                max_idx = i#记录当前最大数的下标
        output = [nums[max_idx]]#第一个
        
        # build output
        for i in range(k, n):
            clean_deque(i)          
            deq.append(i)
            output.append(nums[deq[0]])
        return output
#2.栈模拟双向队列
#164 ms	22.8 MB
from collections import deque
class Solution:
    def maxSlidingWindow(self, nums: 'List[int]', k: 'int') -> 'List[int]':
        queue, res = [], []
        for i in range(len(nums)):
            if len(queue) > 0 and i - queue[0] + 1 > k: del queue[0]
            while len(queue) > 0 and nums[i] > nums[queue[-1]]: del queue[-1]
            queue.append(i)
            if i >= k - 1: 
                res.append(nums[queue[0]])
                
        return res


## 230. 二叉搜索树中第K小的元素
给定一个二叉搜索树，编写一个函数 kthSmallest 来查找其中第 k 个最小的元素。

说明：
你可以假设 k 总是有效的，1 ≤ k ≤ 二叉搜索树元素个数。

示例 1:
```python
输入: root = [3,1,4,null,2], k = 1
   3
  / \
 1   4
  \
   2
输出: 1
示例 2:

输入: root = [5,3,6,2,4,null,null,1], k = 3
       5
      / \
     3   6
    / \
   2   4
  /
 1
输出: 3
```
进阶：
如果二叉搜索树经常被修改（插入/删除操作）并且你需要频繁地查找第 k 小的值，你将如何优化 kthSmallest 函数？

In [ ]:
#1.常规中序
class Solution:
    def kthSmallest(self, root: TreeNode, k: int) -> int:
        stack = []#维护一个栈
        ans = []
        stack.append((0, root))
        while stack:
            opt, node = stack.pop()
            if not node:#已经遍历到子节点了
                continue
            if opt == 1:#优先级标注
                ans.append(node.val)
                if len(ans)==k:
                    return ans[-1]
            else:
                # 只改变这里
                stack.append((0, node.right))
                stack.append((1, node)) 
                stack.append((0, node.left))

## 329. 矩阵中的最长递增路径
给定一个整数矩阵，找出最长递增路径的长度。

对于每个单元格，你可以往上，下，左，右四个方向移动。 你不能在对角线方向上移动或移动到边界外（即不允许环绕）。

示例 1:

输入: nums = 
[
  [9,9,4],
  [6,6,8],
  [2,1,1]
] 
输出: 4 
解释: 最长递增路径为 [1, 2, 6, 9]。
示例 2:

输入: nums = 
[
  [3,4,5],
  [3,2,6],
  [2,2,1]
] 
输出: 4 
解释: 最长递增路径是 [3, 4, 5, 6]。注意不允许在对角线方向上移动。

In [ ]:
#1.dp
#500 ms	14.9 MB
'''
作为预处理，我们必须显式执行拓扑排序。之后，我们可以按照存储的拓扑顺序使用状态转移函数动态地解决问题。

[
  [9,9,4],
  [6,6,8],
  [2,1,1]
] 

dp就是
[
  [4,3,1],
  [3,2,3],
  [2,1,1]
] 

'''
class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        #记忆化dfs/dpdp/优先队列
        if not matrix or not matrix[0]:
            return 0
        #制作一个新的matrix，记录原位置信息和数字
        lst = []
        m = len(matrix)
        n = len(matrix[0])
        for i in range(m):
            for j in range(n):
                lst.append((matrix[i][j],i,j))
        lst.sort()
        
        dp = [[0]*n for _ in range(m)]
        for num,i,j in lst:
            dp[i][j] = 1
            for x,y in [(0,1),(1,0),(0,-1),(-1,0)]:
                idx1 = i+x
                idx2 = j+y
                #dp[i][j]表示以matrix[i][j]结尾的最长递增长度
                if idx1>=0 and idx1<m and idx2>=0 and idx2<n:
                    if matrix[i][j]>matrix[idx1][idx2]:
                        dp[i][j]=max(dp[i][j],1+dp[idx1][idx2])
        return max([dp[i][j] for i in range(m) for j in range(n)])
    
#2.记忆化搜索dfs_手动
class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        if not matrix or not matrix[0]:
            return 0
        m = len(matrix)
        n = len(matrix[0])
        cache = [[0]*n for _ in range(m)]
        def dfs(matrix,i,j,cache):
            if cache[i][j]!=0:
                #visited
                return cache[i][j]
            for x,y in [(0,1),(0,-1),(1,0),(-1,0)]:
                idx1 = i+x
                idx2= j+y
                if idx1>=0 and idx1<m and idx2>=0 and idx2<n:
                    if matrix[i][j]>matrix[idx1][idx2]:
                        cache[i][j] = max(cache[i][j],dfs(matrix,idx1,idx2,cache))
            cache[i][j]+=1
            return cache[i][j]
        ans = 0
        for i in range(m):
            for j in range(n):
                ans = max(ans,dfs(matrix,i,j,cache))
        return ans
#3.修饰器
from functools import lru_cache
class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        if not matrix:
            return 0
        m, n, memo = len(matrix), len(matrix[0]), ((0, 1), (0, -1), (1, 0), (-1, 0))
        @lru_cache(None)
        def dfs(i, j):
            t = 0
            for x, y in memo:
                idx1, idx2 = i + x, j + y
                if 0 <= idx1 < m and 0 <= idx2 < n and matrix[idx1][idx2] > matrix[i][j]:
                    t = max(t, dfs(idx1, idx2))
            return t + 1
        return max(dfs(i, j) for i, j in itertools.product(range(m), range(n)))

In [14]:
def insertsort(nums):
    for i in range(1,len(nums)):
        cur = nums[i]
        j=i-1
        while j>=0 and nums[j]>cur:
            nums[j+1] = nums[j]
            j-=1
        nums[j+1] = cur 
    return nums
nums = [2,1,7,9,5,8]
print(insertsort(nums))

[1, 2, 5, 7, 8, 9]


In [5]:
def bubblesort(nums):
    for i in range(len(nums)-1):
        flag = False
        for j in range(len(nums)-i-1):
            if nums[j]>nums[j+1]:
                nums[j],nums[j+1]=nums[j+1],nums[j]#swap
                flag = True
        if flag == False:
            break
    return nums
nums = [2,1,7,9,5,8]
print(bubblesort(nums))

[1, 2, 5, 7, 8, 9]


In [34]:
def mergesort(array,l,r):
	if l>=r:
		return
	mid = (r-l)//2+l
	mergesort(array,l,mid)
	mergesort(array,mid+1,r)
	merge(array,l,mid,r)
    
def merge(array,l,mid,r):
	i = l
	j = mid+1
	k = l
	while l<=r:
		tmp[l] = array[l]#clone
		l+=1
	while i<=mid and j<=r:
		if array[i]<=array[j]:
			array[k] = tmp[i]
			k+=1
			i+=1
		else:
			array[k] = tmp[j]
			k+=1
			j+=1
	while i<=mid:
		array[k]=tmp[i]
		k+=1
		i+=1
	while j<=r:
		array[k]=tmp[j]
		k+=1
		j+=1
nums = [2,1,7,9,5,8]
mergesort(nums,0,len(nums)-1)
print(nums)

[1, 2, 5, 7, 8, 9]


In [49]:
def quicksort(nums,l,r):
	if l>=r:
		return
	i = l
	j = l
	key = nums[r]
	while j<r:
		if nums[j]<=key:
			nums[i],nums[j] = nums[j],nums[i]#swap
			i+=1
		j+=1
	nums[i],nums[j] = nums[j],nums[i]#swap
	quicksort(nums,l,i-1)
	quicksort(nums,i+1,r)
nums = [2,1,7,9,5,8]
quicksort(nums,0,len(nums)-1)
print(nums)

[1, 2, 5, 7, 8, 9]


## 147. 对链表进行插入排序
对链表进行插入排序。
插入排序的动画演示如上。从第一个元素开始，该链表可以被认为已经部分排序（用黑色表示）。
每次迭代时，从输入数据中移除一个元素（用红色表示），并原地将其插入到已排好序的链表中。


插入排序算法：

插入排序是迭代的，每次只移动一个元素，直到所有元素可以形成一个有序的输出列表。
每次迭代中，插入排序只从输入数据中移除一个待排序的元素，找到它在序列中适当的位置，并将其插入。
重复直到所有输入数据插入完为止。
 
示例 1：

输入: 4->2->1->3
输出: 1->2->3->4
示例 2：

输入: -1->5->3->4->0
输出: -1->0->3->4->5

In [ ]:
class Solution:
    def insertionSortList(self, head: ListNode) -> ListNode:
        #有很多指针
        #1.指向当前已排序的最后一个位置--head
        #2.一个指针每次从表头循环已排序部分--pre=dummy固定表头
        dummy = ListNode(0)
        dummy.next = head
        while head and head.next:
            if head.val<=head.next.val:
                head = head.next
                continue
            #第一个逆序值需要插入
            pre = dummy
            while pre.next.val<head.next.val:
                pre = pre.next
            cur = head.next
            head.next = cur.next
            cur.next = pre.next
            pre.next = cur
        return dummy.next

## 215. 数组中的第K个最大元素
在未排序的数组中找到第 k 个最大的元素。请注意，你需要找的是数组排序后的第 k 个最大的元素，而不是第 k 个不同的元素。

示例 1:

输入: [3,2,1,5,6,4] 和 k = 2
输出: 5
示例 2:

输入: [3,2,3,1,2,4,5,5,6] 和 k = 4
输出: 4
说明:

你可以假设 k 总是有效的，且 1 ≤ k ≤ 数组的长度。

In [ ]:
#1.快排
#2228 ms	19.7 MB
#时间复杂度 : 平均情况O(N)，最坏情况O(N^2)
空间复杂度 : {O}(1)O(1)。
class Solution:
    def findKthLargest(self, nums: List[int], k: int) -> int:
        idx = len(nums)-k
        def quicksort(nums,l,r,idx):
            if l>=r:
                return
            #p = random.randint(l,r)
            i = l
            j = l
            key = nums[r]
            while j<r:
                if nums[j]<=key:
                    nums[i],nums[j] = nums[j],nums[i]#swap
                    i+=1
                j+=1
            nums[i],nums[j] = nums[j],nums[i]#swap
            if i==idx:
                return
            if i>idx:
                quicksort(nums,l,i-1,idx)
            else:
                quicksort(nums,i+1,r,idx)
        quicksort(nums,0,len(nums)-1,idx)
        return nums[idx]
#2.Py小顶堆
#44 ms	14.3 MB
#时间复杂度O(Nlogk)，空间复杂度O(k)
#应该是建一个只能存K个数字的小顶堆，超过K时候，每加进来一个，堆顶就要弹出一个。数组遍历完，最终堆顶的元素就是第K大的
class Solution:
    def findKthLargest(self, nums, k):
        """
        :type nums: List[int]
        :type k: int
        :rtype: int
        """
        return heapq.nlargest(k, nums)[-1]

## 837. 新21点
爱丽丝参与一个大致基于纸牌游戏 “21点” 规则的游戏，描述如下：
爱丽丝以 0 分开始，并在她的得分少于 K 分时抽取数字。 抽取时，她从 [1, W] 的范围中随机获得一个整数作为分数进行累计，其中 W 是整数。 每次抽取都是独立的，其结果具有相同的概率。

当爱丽丝获得不少于 K 分时，她就停止抽取数字。 爱丽丝的分数不超过 N 的概率是多少？

示例 1：

输入：N = 10, K = 1, W = 10
输出：1.00000
说明：爱丽丝得到一张卡，然后停止。
示例 2：

输入：N = 6, K = 1, W = 10
输出：0.60000
说明：爱丽丝得到一张卡，然后停止。
在 W = 10 的 6 种可能下，她的得分不超过 N = 6 分。
示例 3：

输入：N = 21, K = 17, W = 10
输出：0.73278

In [ ]:
##1.dp
#时间复杂度：O(min(N,K+W))。即需要计算的 dp 值的数量 min(N,K+W−1)。

#空间复杂度：O(K+W)。创建了一个长度为 K+W+1的数组dp

class Solution:
    def new21Game(self, N: int, K: int, W: int) -> float:
        if N<K:
            return 0.0
        if K<1:
            return 1.0
        #dpdp?
        #dp[i] : 手中的牌点数为 i 时继续往下抽牌，最终得分x满足 K <= x <= N 的概率
        #手中的牌点数最小为0，最大可能为K - 1 + W（当前持有最大点数K - 1，再抽一张最大的牌W）
        dp = [0]*(K-1+W+1+1)
        '''
        该init了：
        手中牌为 K to N 时满足 K <= i <= N 的概率为 1
        手中牌为 (N + 1) to (K - 1 + W) 时满足 K <= i <= N 的概率为 0
        '''
        for i in range(K,N+1):
            dp[i]=1.0
        summ = (N-K+1)-1.0 #可能取到的位置数量
        for i in range(K-1,-1,-1):
            #可能抽到的牌有W张(点数为[1,W])，且抽到这W张牌的概率相等
            summ = summ-dp[i+W+1]+dp[i+1]
            dp[i] = summ/W
        return dp[0]


## 344. 反转字符串
编写一个函数，其作用是将输入的字符串反转过来。输入字符串以字符数组 char[] 的形式给出。

不要给另外的数组分配额外的空间，你必须原地修改输入数组、使用 O(1) 的额外空间解决这一问题。

你可以假设数组中的所有字符都是 ASCII 码表中的可打印字符。

 

示例 1：

输入：["h","e","l","l","o"]  
输出：["o","l","l","e","h"]  
示例 2：  

输入：["H","a","n","n","a","h"]  
输出：["h","a","n","n","a","H"]  


In [ ]:
#1.递归 O(n),O(n)
#60 ms	45.4 MB
class Solution:
    def reverseString(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        def helper(s,i,j):
            if i<j:
                s[i],s[j] = s[j],s[i]
                helper(s,i+1,j-1)
        helper(s,0,len(s)-1)
#2.迭代 O(n),O(1)
#52 ms	14.6 MB
class Solution:
    def reverseString(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        i = 0
        j = len(s)-1
        while i<j:
            s[i],s[j] = s[j],s[i]
            i+=1
            j-=1 
        

## 24. 两两交换链表中的节点
给定一个链表，两两交换其中相邻的节点，并返回交换后的链表。
你不能只是单纯的改变节点内部的值，而是需要实际的进行节点交换。

 

示例:

给定 1->2->3->4, 你应该返回 2->1->4->3.


In [ ]:
#1.递归
#40 ms	13.5 MB
class Solution:
    def swapPairs(self, head: ListNode) -> ListNode:
        '''
        我们可以定义函数 swap(head) 以实现解决方案，其中输入的参数 head 指向链表的头节点。而该函数应当返回将链表中每两个相邻节点交换后得到的新列表的头节点 head 。按照我们上面列出的步骤，我们可以按下面的流程来实现函数：
首先，我们交换列表中的前两个节点，也就是 head 和 head.next；
然后我们以 swap(head.next.next) 的形式调用函数自身，以交换头两个节点之后列表的其余部分。
最后，我们将步骤（2）中的子列表的返回头与步骤（1）中交换的两个节点相连，以形成新的链表。
作为练习，你可以尝试使用我们上面提供的步骤来实施解决方案。单击 “两两交换链表中的节点” 以尝试自己的解决方案。
        '''
        if not head or not head.next:
            return head
        next = head.next
        head.next = self.swapPairs(next.next)
        next.next = head
        return next
#2.迭代
#44 ms	13.7 MB
class Solution:
    def swapPairs(self, head: ListNode) -> ListNode:
        dummy = ListNode(0)
        dummy.next = head
        p = dummy
        while p.next and p.next.next:
            next = p.next
            p.next = next.next
            next.next = next.next.next
            p.next.next = next
            p = next
        return dummy.next

## 118. 杨辉三角
给定一个非负整数 numRows，生成杨辉三角的前 numRows 行。
在杨辉三角中，每个数是它左上方和右上方的数的和。

示例:
```python
输入: 5
输出:
[
     [1],
    [1,1],
   [1,2,1],
  [1,3,3,1],
 [1,4,6,4,1]
]
```

In [ ]:
#1.递归
class Solution:
    def generate(self, numRows: int) -> List[List[int]]:
        ans = []
        for i in range(numRows):
            #init
            tmp = [0]*(i+1)
            tmp[0]=1
            tmp[-1]=1
            ans.append(tmp)
        def helper(depth,ans):
            if depth>=numRows:
                return
            for i in range(1,depth):
                ans[depth][i] = ans[depth-1][i-1]+ans[depth-1][i]
            helper(depth+1,ans)
        if numRows<=2:
            return ans
        helper(2,ans)
        return ans
#2.杨辉三角2
class Solution:
    def getRow(self, rowIndex: int) -> List[int]:
        ans = []
        numRows = rowIndex+1
        if numRows<2:
            return [1]
        if numRows==2:
            return [1,1]
        self.ans = [1,1]
        def helper(depth):
            if depth>=numRows:
                return ans
            tmp = [0]*(depth+1)
            tmp[0]=1
            tmp[-1]=1
            for i in range(1,depth):
                tmp[i] = self.ans[i-1]+self.ans[i]
            self.ans = tmp
            helper(depth+1)
        helper(2)
        return self.ans

## 汉诺塔

In [64]:
def hano(a,b,c,n):
	if n>0:
		hano(a,c,b,n-1)
		print(a+'->'+c)
		hano(b,a,c,n-1)
hano('a','b','c',3)

a->c
a->b
c->b
a->c
b->a
b->c
a->c


## 91. 解码方法
一条包含字母 A-Z 的消息通过以下方式进行了编码：

'A' -> 1
'B' -> 2
...
'Z' -> 26
给定一个只包含数字的非空字符串，请计算解码方法的总数。

示例 1:

输入: "12"
输出: 2
解释: 它可以解码为 "AB"（1 2）或者 "L"（12）。
示例 2:

输入: "226"
输出: 3
解释: 它可以解码为 "BZ" (2 26), "VF" (22 6), 或者 "BBF" (2 2 6) 。

In [ ]:
#1.递归
class Solution:
    def numDecodings(self, s: str) -> int:
        s = list(s)
        if not s or s[0]=='0':
            return 0
        @lru_cache(None)
        def helper(idx):
            if idx<=0:
                return 1
            cnt = 0
            cur = s[idx]
            pre = s[idx-1]
            if cur>'0':
                #先统计单位的
                cnt = helper(idx-1)
            if pre=='1' or (pre=='2' and cur<='6'):
                #两位的,此处重复计算过多导致递归超时，带了个修饰器稳了
                cnt += helper(idx-2)
            return cnt
        return helper(len(s)-1)
#2.dp
#痛苦边界处理
class Solution:
    def numDecodings(self, s: str) -> int:
        cnt = 0
        if len(s)==0 or len(s)==1 and s[0]=='0':
            return 0
        if len(s)==1:
            return 1
        dp = [0]*(len(s)+2)
        dp[0]=1
        dp[1]=1
        s = "99" + s 
        for i in range(2,len(s)):
            if s[i-1]=='1' or s[i-1]=='2' and s[i]<='6':
                dp[i]+=dp[i-2]
            if s[i]!='0':
                dp[i]+=dp[i-1]
        return dp[-1]

## 779. 第K个语法符号
在第一行我们写上一个 0。接下来的每一行，将前一行中的0替换为01，1替换为10。

给定行数 N 和序数 K，返回第 N 行中第 K个字符。（K从1开始）


例子:

输入: N = 1, K = 1
输出: 0

输入: N = 2, K = 1
输出: 0

输入: N = 2, K = 2
输出: 1

输入: N = 4, K = 5
输出: 1

解释:
第一行: 0
第二行: 01
第三行: 0110
第四行: 01101001

In [ ]:
#1.规律递归1——父节点
#什么都不想的遍历性递归会超时，从下往上看递归找父节点值及规律
#暴力的话时间空间都是O(2^n)，这个变体只需要找到父节点走一个高度O(n)
class Solution:
    def kthGrammar(self, N: int, K: int) -> int:
        if N==1 and K==1:
            return 0
        a = self.kthGrammar(N-1,(K+1)//2)
        b = -(a-1)#若a=0则b=1,若a=1则b=0
        if K%2==1:
            #K为奇数则其值与父结点相同
            return a
        else:
            return b
#2.规律递归2——翻转
#后半部分总是与前半部分相反，前半部分是 '0'，后半部分变成 '1'，前半部分是 '1'，后半部分变成 '0'。
class Solution(object):
    def kthGrammar(self, N, K):
        if N == 1: return 0
        if K <= (2**(N-2)):
            #K在前半，回上一行找
            return self.kthGrammar(N-1, K)
        return self.kthGrammar(N-1, K - 2**(N-2)) ^ 1
    #反之找前半的镜像位置，还得取反数
#3.规律递归3——
#N位二进制数的第二部分第一个数一定是2**(N-1)，二进制是10...0，因此所有第二部分数的第一个bit一定是1。
#所以K-1的二进制数里的所有的1都是翻转的痕迹。
class Solution(object):
    def kthGrammar(self, N, K):
        return bin(K - 1).count('1') % 2

## 39. 组合总和
给定一个无重复元素的数组 candidates 和一个目标数 target ，找出 candidates 中所有可以使数字和为 target 的组合。

candidates 中的数字可以无限制重复被选取。

说明：

所有数字（包括 target）都是正整数。
解集不能包含重复的组合。 
示例 1:

输入: candidates = [2,3,6,7], target = 7,
所求解集为:
[
  [7],
  [2,2,3]
]
示例 2:

输入: candidates = [2,3,5], target = 8,
所求解集为:
[
  [2,2,2,2],
  [2,3,3],
  [3,5]
]

In [ ]:
#1.回溯
class Solution:
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        ans = []
        n = len(candidates)
        def helper(candidates,target,start,tmp,ans):
            if target<0 or start==n:
                return
            if target==0:
                ans.append(tmp)
                return
            for i in range(start,len(candidates)):
                helper(candidates,target-candidates[i],i,tmp+[candidates[i]],ans)
        helper(candidates,target,0,[],ans)
        return ans

## 52. N皇后 II
皇后，是国际象棋中的棋子，意味着国王的妻子。皇后只做一件事，那就是“吃子”。当她遇见可以吃的棋子时，就迅速冲上去吃掉棋子。
当然，她横、竖、斜都可走一或七步，可进可退。（引用自 百度百科 - 皇后 ）
n 皇后问题研究的是如何将 n 个皇后放置在 n×n 的棋盘上，并且使皇后彼此之间不能相互攻击。

In [ ]:
#1.回溯
class Solution:
    def totalNQueens(self, n: int) -> int:
        self.cnt = 0
        def helper(n,row,columns):
            if row==n:
                #遍历到最低端而且合法了，发现一种新方法
                self.cnt+=1
                return
            for col in range(n):
                columns[row]=col#表示当前行所在的列
                if check(row,col,columns):
                    helper(n,row+1,columns)
                columns[row]=-1 #不合法则不标记且直接回溯

        def check(row,col,columns):
            for r in range(row):
                if columns[r]==col or (row-r)==abs(col-columns[r]):
                    #在同一列和对角线上都不行
                    return False
            return True

        helper(n,0,[0]*n)
        return self.cnt
#2.位运算
#44 ms	13.7 MB
'''
使用常规深度优先一层层搜索
使用三个整形分别标记每一层哪些格子可以放置皇后，这三个整形分别代表列、左斜下、右斜下（_col, ld, rd_），
二进制位为1代表不能放置，0代表可以放置
核心两个位运算：
x & -x 代表除最后一位1保留，其它位全部为0 #lowbit
x & (x - 1) 代表将最后一位1变成0 #如果最低位是0，那么减1后，这个0就变为了1，并且向高位借位，直到遇到第一个1，
这个1会因为之前的借位变成0，此时借位清0，更高位的数据就不会发生变化。这样按位与后，最低位的1就变成了0，而其他的数并没有变化。
'''
class Solution:
    def totalNQueens(self, n: int) -> int:
        if n<1:
            return 0
        self.res = 0
        def dfs(n,row,col,ld,rd):
            if row >= n:
                self.res+=1
                return
            bits = ~(col | ld | rd) & ((1 << n) - 1)#获取当前空位，标记为1
            #row，ld，rd进行“或”运算，求得所有可以放置皇后的列，对应位为0，然后再取反“与”上全1的数，此时1代表可以放，0代表不可以放。
            while bits > 0:#bit大于0说明还有可以放置的位置
                pick = bits & -bits #lowbit操作，找到最低位的1，获取每行最后一个可以放置的位置
                dfs(n, row + 1, col | pick, (ld | pick) << 1, (rd | pick) >> 1)
                #col | pick 表示 pick 位被占用
                #(ld | pick) << 1 表示左下往斜左下移一位，占用
                # (rd | pick) >> 1 表示右下往斜右下移一位，占用
                bits &= bits - 1 #去掉最低位的1
        dfs(n,0,0,0,0)
        return self.res

## 567. 字符串的排列
给定两个字符串 s1 和 s2，写一个函数来判断 s2 是否包含 s1 的排列。

换句话说，第一个字符串的排列之一是第二个字符串的子串。

示例1:

输入: s1 = "ab" s2 = "eidbaooo"
输出: True
解释: s2 包含 s1 的排列之一 ("ba").
 

示例2:

输入: s1= "ab" s2 = "eidboaoo"
输出: False
 

注意：

输入的字符串只包含小写字母
两个字符串的长度都在 [1, 10,000] 之间


In [ ]:
#1.list模拟hash+滑窗
#9532 ms	13.6 MB
#AC了，险些超时
class Solution:
    def checkInclusion(self, s1: str, s2: str) -> bool:
        #比起dict明显是list要好操作一点
        n = len(s1)
        i = 0
        j = n
        memo = [0]*26
        for l in s1:
            #计数器
            memo[ord(l)-ord('a')]+=1
        #滑窗滑窗
        while j<=len(s2):
            tmp = [0]*26
            flag = 0
            for l in s2[i:j]:
                if memo[ord(l)-ord('a')]==0:
                    #滑窗滑动
                    flag = 1
                    break
                else:
                    tmp[ord(l)-ord('a')]+=1
            #print(tmp)
            if flag==0 and tmp==memo:
                return True
            i+=1
            j+=1
        return False

#2.优化滑窗
#速度激增
#76 ms	13.6 MB
class Solution:
    def checkInclusion(self, s1: str, s2: str) -> bool:
        #比起dict明显是list要好操作一点
        if len(s1)>len(s2):
            return False
        n = len(s1)
        memo = [0]*26
        tmp = [0]*26
        for i in range(n):
            #计数器
            memo[ord(s1[i])-ord('a')]+=1
            tmp[ord(s2[i])-ord('a')]+=1
        #滑窗滑窗
        for j in range(n,len(s2)):
            if tmp==memo:
                return True
            #滑窗滑动,更新边界，而不是每次更新一个字典
            tmp[ord(s2[j-n])-ord('a')]-=1
            tmp[ord(s2[j])-ord('a')]+=1
        #整个算法采用左闭右开区间，因此最后还有一个窗口没有判断
        return tmp==memo
#3.再次优化——变量统计法
#124 ms	13.8 MB
class Solution(object):
    def checkInclusion(self, s1, s2):
        """
        :type s1: str
        :type s2: str
        :rtype: bool
        """
        l1, l2 = len(s1), len(s2)
        c1 = collections.Counter(s1)
        c2 = collections.Counter()
        cnt = 0 #统计变量，全部26个字符，频率相同的个数，当cnt==s1字母的个数的时候，就是全部符合题意，返回真
        p = q = 0 #滑动窗口[p,q]
        while q < l2:
            c2[s2[q]] += 1
            if c1[s2[q]] == c2[s2[q]]: #对于遍历到的字母，如果出现次数相同
                cnt += 1               #统计变量+1
            if cnt == len(c1):         #判断结果写在前面，此时证明s2滑动窗口和s1全部字符相同，返回真
                return True
            q += 1                     #滑动窗口右移
            if q - p + 1 > l1:         #这是构造第一个滑动窗口的特殊判断，里面内容是维护边界滑动窗口
                if c1[s2[p]] == c2[s2[p]]:    #判断性的if写在前面，因为一旦频率变化，这个统计变量就减1
                    cnt -= 1
                c2[s2[p]] -= 1                #字典哈希表移除最前面的字符
                if c2[s2[p]] == 0:            #由于counter特性，如果value为0，必须删除它
                    del c2[s2[p]]
                p += 1                        #最前面的下标右移动
        return False

## 43. 字符串相乘
给定两个以字符串形式表示的非负整数 num1 和 num2，返回 num1 和 num2 的乘积，它们的乘积也表示为字符串形式。

示例 1:

输入: num1 = "2", num2 = "3"
输出: "6"
示例 2:

输入: num1 = "123", num2 = "456"
输出: "56088"
说明：

num1 和 num2 的长度小于110。
num1 和 num2 只包含数字 0-9。
num1 和 num2 均不以零开头，除非是数字 0 本身。
不能使用任何标准库的大数类型（比如 BigInteger）或直接将输入转换为整数来处理。

In [ ]:
#1.按位计算
#148 ms	13.6 MB
class Solution:
    def multiply(self, num1: str, num2: str) -> str:
        #是按位乘结果叠加吗
        '''
                     0 1 2 3 4  
              
                        1 2 3
                    *     4 5
                    ---------
                          1 5
                        1 0
                      0 5
                    ---------
                      0 6 1 5
                        1 2
                      0 8
                    0 4
                    ---------
                    0 5 5 3 5
        '''
        #位数
        n1 = len(num1)
        n2 = len(num2)
        if n1<0 or n2<0:
            return ""
        ans = [0]*(n1+n2)#怕有进位
        for i in range(n1-1,-1,-1):
            for j in range(n2-1,-1,-1):
                #从低位到高位
                cur = (ord(num1[i])-ord('0'))*(ord(num2[j])-ord('0'))
                cur+=ans[i+j+1]#上一位计算结果
                ans[i+j] += cur//10
                ans[i+j+1] = cur%10
        ans = list(map(str,ans))
        for k in range(len(ans)):
            if ans[k]!='0':
                break
        return "".join(ans[k:])
            

## 128. 最长连续序列
给定一个未排序的整数数组，找出最长连续序列的长度。

要求算法的时间复杂度为 O(n)。

示例:

输入: [100, 4, 200, 1, 3, 2]
输出: 4
解释: 最长连续序列是 [1, 2, 3, 4]。它的长度为 4。

In [ ]:
#1.hash
'''
用哈希表存储每个端点值对应连续区间的长度
若数已在哈希表中：跳过不做处理
若是新数加入：
取出其左右相邻数已有的连续区间长度 left 和 right
计算当前数的区间长度为：cur_length = left + right + 1
根据 cur_length 更新最大长度 max_length 的值
更新区间两端点的长度值
'''
class Solution:
    def longestConsecutive(self, nums: List[int]) -> int:
        memo = {}
        maxlen = 0
        for i in range(len(nums)):
            #不会有重复数,不会重叠
            if nums[i] not in memo:
                #实际上只看端点的值
                left = memo.get(nums[i]-1,0)
                right = memo.get(nums[i]+1,0)

                curlen = 1+left+right
                if curlen>maxlen:
                    maxlen = curlen
                memo[nums[i]] = curlen
                memo[nums[i]-left] = curlen #区间最左边
                memo[nums[i]+right] = curlen #区间最右边
        return maxlen 

#2.并查集
class Solution:
	def longestConsecutive(self, nums: List[int]) -> int:
		n = len(nums)
		if n==0:
			return 0
		self.uf = dict([(num,num) for num in nums])
		self.rank = collections.defaultdict(lambda: 1)
		self.cnt = collections.defaultdict(lambda: 1)
		self.ans = 1
		for num in nums:
			if num-1 in self.uf:
				self.union(num-1,num) 
		return self.ans

	def find(self,p):
		if self.uf[p]==p:
			#root
			return p
		self.uf[p] = self.find(self.uf[p]) #路径压缩
		return self.uf[p]

	def union(self,x,y):
		xroot = self.find(x)
		yroot = self.find(y)
		if xroot==yroot:
			return
		elif self.rank[xroot]>self.rank[yroot]:
			#负数比较
			self.cnt[xroot]+=self.cnt[yroot]
			self.uf[yroot] = xroot
			self.ans = max(self.ans,abs(self.cnt[xroot]))
		else:
			self.cnt[yroot] += self.cnt[xroot]
			self.uf[xroot] = yroot
			if self.rank[xroot]==self.rank[yroot]:
				self.rank[yroot] +=1
			self.ans = max(self.ans,abs(self.cnt[yroot]))
    

## 300. 最长上升子序列
给定一个无序的整数数组，找到其中最长上升子序列的长度。

示例:

输入: [10,9,2,5,3,7,101,18]
输出: 4 
解释: 最长的上升子序列是 [2,3,7,101]，它的长度是 4。
说明:

可能会有多种最长上升子序列的组合，你只需要输出对应的长度即可。
你算法的时间复杂度应该为 O(n2) 。
进阶: 你能将算法的时间复杂度降低到 O(n log n) 吗?

In [ ]:
#1.dp_O(n^2)
#1344 ms	13.9 MB	
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        #dpdp
        #dp[i]存储以nums[i]结尾的最长上升子序列长度
        #f[i] = max(f[i-1])+1 #设定算上自己，这个式子仅当nums[i-1]<nums[i]的时候成立
        maxlen = 1
        if len(nums)<1:
            return 0
        dp = [1]*len(nums)
        for i in range(1,len(nums)):
            for j in range(i):
                if nums[i]>nums[j]:
                    dp[i] = max(dp[i],dp[j]+1) #递增序列有扩大
            if dp[i]>maxlen:
                maxlen = dp[i]
        #print(dp)
        return maxlen
#2.二分——O(nlogn)
#72 ms	13.8 MB
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        #二分
        #dp[i]: 所有长度为i+1的递增子序列中, 最小的那个序列尾数.
        #对数组进行迭代, 依次判断每个数num将其插入dp数组相应的位置
        '''
        1. num > dp[maxL], 表示num比所有已知递增序列的尾数都大, 将num添加入dp
           数组尾部, 并将最长递增序列长度maxL加1
        2. dp[i-1] < num <= dp[i], 只更新相应的dp[i]
        '''
        maxlen = 0
        dp =[1]*len(nums)
        for num in nums:
            l = 0
            r = maxlen
            while l<r:
                mid = (r-l)//2+l
                if dp[mid]<num:
                    l = mid+1
                else:
                    r = mid
            dp[l]=num #l！=maxlen的话，找到插入位置，把对应长度的尾结点更新成该点
            #l==maxlen的话就是数组的最尾部，此时maxlen需要更新
            if l==maxlen:
                maxlen+=1
        return maxlen

## 516. 最长回文子序列
给定一个字符串s，找到其中最长的回文子序列，并返回该序列的长度。可以假设s的最大长度为1000。

示例 1:
输入:

"bbbab"
输出:4
一个可能的最长回文子序列为 "bbbb"。

示例 2:
输入:

"cbbd"
输出:2


In [ ]:
#1.dpdp
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        #不是求连续的回文子串，而是子序列
        #dpdp
        #dp[i][j]表示idx从i到j的最长回文长度，然后比对两边看还可不可以扩大
        #相同时递推公式：dp[0][n-1] = dp[1][n-2]+2
        #不同时递推公式：dp[0][n-1] = max(dp[1][n-1],dp[0][n-2])
        #return dp[0][n-1]
        n = len(s)
        dp = [[0]*n for _ in range(n)] #n*n
        #init
        for i in range(n):
            dp[i][i] = 1
        for l in range(2,n+1):
            #将搜索覆盖字符长度逐步扩大
            for i in range(0,n-l+1):
                j = i+l-1 #起始和停止位置
                if s[i]==s[j]:
                    dp[i][j] = dp[i+1][j-1]+2 if l>2 else 2 #刚好长度为2的时候,中间没有回文串了
                else:
                    dp[i][j] = max(dp[i+1][j],dp[i][j-1])
        return dp[0][n-1]
#2.记忆化递归
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        import functools
        @functools.lru_cache(None)
        def f(s):
            if len(s) <= 1:
                return len(s)
            if s[0] == s[-1]:
                return f(s[1:-1]) + 2
            else:
                return max(
                        f(s[1:]),
                        f(s[:-1]))
        return f(s)
#3.单维dp
'''
在计算dp[i][j]（i不等于j）时,只用到其左边、下边和左下边信息。
若改用一维数组记录信息，其左边和下边信息在计算时仍保存在数组里，因此，只需用一变量temp记录会发生变化的左下边信息即可（在其变化前）。

'''
class Solution:
    def longestPalindromeSubseq(self, s: str) -> int:
        n = len(s)
        dp = [0]*n #单维n
        #init
        for i in range(n):
            dp[i]=1
            maxl=0
        #tmp存储的是上一轮的dp值，也就是dp[i+1][j-1]也就是左下侧
        #逆序是沿着i的方向从下至上打一遍，沿着j从左往右打一遍，所以正下和正左都是不变的，[正左（已更新），正下（未更新）]
        for j in range(i+1, n):
                if s[i] == s[j]:
                    #aa = dp[j]                    
                    #dp[j] = temp + 2
                    #temp = aa
                    temp, dp[j] = dp[j], temp + 2
                else:
                    temp = dp[j]#如果是下侧大则不用更新
                    if dp[j-1] > dp[j]:
                        dp[j] = dp[j-1]#不然就是左侧的大
        return dp[-1] #即为dp[0][n-1]


## 1466. 重新规划路线
n 座城市，从 0 到 n-1 编号，其间共有 n-1 条路线。因此，要想在两座不同城市之间旅行只有唯一一条路线可供选择（路线网形成一颗树）。去年，交通运输部决定重新规划路线，以改变交通拥堵的状况。

路线用 connections 表示，其中 connections[i] = [a, b] 表示从城市 a 到 b 的一条有向路线。

今年，城市 0 将会举办一场大型比赛，很多游客都想前往城市 0 。

请你帮助重新规划路线方向，使每个城市都可以访问城市 0 。返回需要变更方向的最小路线数。

题目数据 保证 每个城市在重新规划路线方向后都能到达城市 0 。

示例 1：
输入：n = 6, connections = [[0,1],[1,3],[2,3],[4,0],[4,5]]
输出：3
解释：更改以红色显示的路线的方向，使每个城市都可以到达城市 0 。

输入：n = 5, connections = [[1,0],[1,2],[3,2],[3,4]]
输出：2
解释：更改以红色显示的路线的方向，使每个城市都可以到达城市 0 。

In [ ]:
#1.邻接表--建图bfs
#累加逆向边
class Solution:
    def minReorder(self, n: int, connections: List[List[int]]) -> int:
        '''
        5<-4->0->1->3->2
        思路是把0作为bfs的第一个节点提起来当树root，遍历的时候把路改为指向当前node节点，不需要改就把自己加到队列
        需要改把next添加到队列里，直到树遍历完，需要visited
        '''
        conn_idx = [[] for _ in range(n)] #n*n
        for i in range(len(connections)):
            #i就当路线号
            conn_idx[connections[i][0]].append(i)
            conn_idx[connections[i][1]].append(i)
        
        visited = [False]*len(connections)
        ans = 0

        q = collections.deque()
        q.append(0)

        while q:
            node = q.popleft()
            for idx in conn_idx[node]:
                if visited[idx]:
                    continue
                visited[idx] = True
                #idx是路线号
                a = connections[idx][0]
                b = connections[idx][1]

                ans += (a == node) #这路线是node起头
                a = b if a==node else a
                q.append(a)
        return ans

#2.从0开始dfs，累加中间遇到的逆向边
#from typing import List
class Solution:
    def minReorder(self, n: int, connections: List[List[int]]) -> int:
        link = [[] for _ in range(n)]
        edges = set()

        for a, b in connections:
            #邻接表
            link[a].append(b)
            link[b].append(a)
            edges.add((a, b))

        ans = [0]
        def dfs(cur, prev):
            for next in link[cur]:
                if next != prev:#不是已经看过的，not visited
                    if (next, cur) not in edges:
                        #这就是逆序了
                        ans[0] += 1

                    dfs(next, cur)

        dfs(0, 0x7fffffff)
        return ans[0]

## 126. 单词接龙 II
给定两个单词（beginWord 和 endWord）和一个字典 wordList，找出所有从 beginWord 到 endWord 的最短转换序列。转换需遵循如下规则：

每次转换只能改变一个字母。
转换过程中的中间单词必须是字典中的单词。
说明:

如果不存在这样的转换序列，返回一个空列表。
所有单词具有相同的长度。
所有单词只由小写字母组成。
字典中不存在重复的单词。
你可以假设 beginWord 和 endWord 是非空的，且二者不相同。
示例 1:

输入:
beginWord = "hit",
endWord = "cog",
wordList = ["hot","dot","dog","lot","log","cog"]

输出:
[
  ["hit","hot","dot","dog","cog"],
  ["hit","hot","lot","log","cog"]
]
示例 2:

输入:
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log"]

输出: []

解释: endWord "cog" 不在字典中，所以不存在符合要求的转换序列。

In [ ]:
#1.单向bfs
#1848 ms	15.7 MB
from collections import defaultdict
from typing import List
from collections import deque
import string


class Solution:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        # 先将 wordList 放到哈希表里，便于判断某个单词是否在 wordList 里
        word_set = set(wordList) #set是hashmap的一种形式，和dict类似，也是一组key的集合，但不存储value。
        res = []
        if len(word_set) == 0 or endWord not in word_set:
            return res
        #使用 BFS 的同时记录遍历的路径，形式：哈希表。哈希表的 key 记录了「顶点字符串」，
        #哈希表的值 value 记录了 key 对应的字符串在广度优先遍历的过程中得到的所有后继结点列表 successors；
        successors = defaultdict(set)
        # 第 1 步：使用广度优先遍历得到后继结点列表 successors
        # key：字符串，value：广度优先遍历过程中 key 的后继结点列表
        found = self.__bfs(beginWord, endWord, word_set, successors)
        if not found:
            return res
        # 第 2 步：基于后继结点列表 successors ，使用回溯算法得到所有最短路径列表
        path = [beginWord]
        self.__dfs(beginWord, endWord, successors, path, res)
        return res

    def __bfs(self, beginWord, endWord, word_set, successors):
        queue = deque()
        queue.append(beginWord)
        visited = set()
        visited.add(beginWord)

        found = False
        word_len = len(beginWord)
        next_level_visited = set()

        while queue:
            current_size = len(queue)
            for i in range(current_size):
                current_word = queue.popleft()
                word_list = list(current_word) #拆出字母

                for j in range(word_len):
                    origin_char = word_list[j]
                    #只替换一个字母————找路

                    for k in string.ascii_lowercase:
                        word_list[j] = k#要尝试这个位置所有的可能性
                        next_word = ''.join(word_list)

                        if next_word in word_set:
                            if next_word not in visited:
                                if next_word == endWord:
                                    found = True #不能现在就跳出，可能有多个
                                next_level_visited.add(next_word)#每一层使用一个新的 nextLevelVisited 哈希表
                                #记录当前层的下一层可以访问到哪些结点，直到上一层队列里的值都出队以后，需要检查当前队列中未出队的单词是否还能与wordA建立联系；
                                #nextLevelVisited 哈希表才添加到总的 visited 哈希表，这样记录当前结点和广度优先遍历到的子结点列表才不会遗漏。
                                queue.append(next_word)

                                successors[current_word].add(next_word)#字典：key-value，添加路径
                    word_list[j] = origin_char#变回去
            if found:
                break
            # 取两集合全部的元素（并集，等价于将 next_level_visited 里的所有元素添加到 visited 里）
            visited |= next_level_visited
            next_level_visited.clear()
        return found

    def __dfs(self, beginWord, endWord, successors, path, res):
        if beginWord == endWord:
            res.append(path[:])#新的一条路
            return

        if beginWord not in successors:
            return

        successor_words = successors[beginWord]
        for next_word in successor_words:
            path.append(next_word)
            self.__dfs(next_word, endWord, successors, path, res)
            path.pop()


if __name__ == '__main__':
    beginWord = "hit"
    endWord = "cog"
    wordList = ["hot", "dot", "dog", "lot", "log", "cog"]

    solution = Solution()
    res = solution.findLadders(beginWord, endWord, wordList)
    print(res)
    
## 2.双向bfs
#80 ms	14.3 MB
#只是bfs部分改了
def __bidirectional_bfs(self, beginWord, endWord, word_set, successors):
        visited = set()
        visited.add(beginWord)
        visited.add(endWord)

        begin_visited = set()
        begin_visited.add(beginWord)

        end_visited = set()
        end_visited.add(endWord)

        found = False
        forward = True
        word_len = len(beginWord)
        while begin_visited:
            if len(begin_visited) > len(end_visited):
                #取小的那个开始遍历，统计学意义上搜索量更小
                begin_visited, end_visited = end_visited, begin_visited
                forward = not forward

            next_level_visited = set()
            for current_word in begin_visited:
                word_list = list(current_word)
                for j in range(word_len):
                    origin_char = word_list[j]
                    for k in string.ascii_lowercase:
                        word_list[j] = k
                        next_word = ''.join(word_list)
                        if next_word in word_set:
                            if next_word in end_visited:
                                found = True
                                # 在另一侧找到单词以后，还需把这一层关系添加到「后继结点列表」
                                self.__add_to_successors(successors, forward, current_word, next_word)
                            if next_word not in visited:
                                next_level_visited.add(next_word)
                                self.__add_to_successors(successors, forward, current_word, next_word)
                    word_list[j] = origin_char
            begin_visited = next_level_visited
            # 取两集合全部的元素（并集，等价于将 next_level_visited 里的所有元素添加到 visited 里）
            visited |= next_level_visited
            if found:
                break
        return found

    def __add_to_successors(self, successors, forward, current_word, next_word):
        #curword和nextword对互相来讲是nextword
        if forward:
            successors[current_word].add(next_word)
        else:
            successors[next_word].add(current_word)

In [ ]:
# 牛客字节2020算法第一批
#输入格式
n = int(input())#行数
grid = []
for _ in range(n):
    s = input()
    if s!='':
        #temp = [j for j in s.strip().split()] #str输入
        temp = [int(j) for j in s.strip().split()] #int输入
        grid.append(temp)
    else:
        break
grid = sorted(grid,key = lambda x: x[0])
res = []
for i in range(n):
    for j in range(i+1,n):
    #和x大的比较
        if grid[j][1]>grid[i][1]:
            break
    else:
        res.append((grid[i][0],grid[i][1]))
for i in res:
    print(str(i[0])+' '+str(i[1]))

In [80]:
def aa(n,grid):
    res = []
    grid = sorted(grid,key = lambda x: x[0])
    for i in range(n):
        for j in range(i+1,n):
        #和x大的比较
            if grid[j][1]>grid[i][1]:
                break
        else:
            res.append((grid[i][0],grid[i][1]))
    print(res)
n = 5
grid = [[1,2],[4,7],[6,3],[7,2],[2,3]]
aa(n,grid)

[(4, 7), (6, 3), (7, 2)]


## 71. 简化路径
以 Unix 风格给出一个文件的绝对路径，你需要简化它。或者换句话说，将其转换为规范路径。

在 Unix 风格的文件系统中，一个点（.）表示当前目录本身；此外，两个点 （..） 表示将目录切换到上一级（指向父目录）；两者都可以是复杂相对路径的组成部分。更多信息请参阅：Linux / Unix中的绝对路径 vs 相对路径
请注意，返回的规范路径必须始终以斜杠 / 开头，并且两个目录名之间必须只有一个斜杠 /。最后一个目录名（如果存在）不能以 / 结尾。此外，规范路径必须是表示绝对路径的最短字符串。

 

示例 1：

输入："/home/"
输出："/home"
解释：注意，最后一个目录名后面没有斜杠。
示例 2：

输入："/../"
输出："/"
解释：从根目录向上一级是不可行的，因为根是你可以到达的最高级。
示例 3：

输入："/home//foo/"
输出："/home/foo"
解释：在规范路径中，多个连续斜杠需要用一个斜杠替换。
示例 4：

输入："/a/./b/../../c/"
输出："/c"
示例 5：

输入："/a/../../b/../c//.//"
输出："/c"
示例 6：

输入："/a//b////c/d//././/.."
输出："/a/b/c"

In [ ]:

class Solution:
    def simplifyPath(self, path: str) -> str:
        #1.规范化：遇到//跳过
        #2.简化：遇到/./则删除为/，遇到/..则往回找
        #维护一个栈，每遇到一个/
        tmp = []
        l=1
        for part in path.split('/'):
            #print(part)
            if part not in ['.','..','']:
                #是正常的内容
                tmp.append(part)
            elif part=='..' and tmp:
                tmp.pop()#一下弹出一趴
            
        return '/'+'/'.join(tmp)
            


## 93. 复原IP地址
给定一个只包含数字的字符串，复原它并返回所有可能的 IP 地址格式。

有效的 IP 地址正好由四个整数（每个整数位于 0 到 255 之间组成），整数之间用 '.' 分隔。

 

示例:

输入: "25525511135"
输出: ["255.255.11.135", "255.255.111.35"]


In [ ]:
class Solution:
    def restoreIpAddresses(self, s: str) -> List[str]:
        #拆分为0-255之间的4个int
        #回溯法吗？找不到了就返回
        n = len(s)
        if n/4<1 or n/4>3:
            return ''
        self.ans = []
        def helper(start,tmp):
            #print(tmp)
            if start==len(s) and len(tmp)==4:
                self.ans.append(".".join(tmp))
                return
            if len(tmp)<4 and start<len(s):
                for i in range(1,4):
                    #print(start,start+i)
                    #if s[start:start+i]!='':
                    if int(s[start:start+i])<=255 and str(int(s[start:start+i]))==s[start:start+i]:
                        #剪枝，处理0开头字符串
                        #print(1)
                        helper(start+i,tmp+[s[start:start+i]])

        helper(0,[])
        return self.ans
